In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
base_path = './lfw/'

In [3]:
from tqdm import tqdm

In [47]:
images = []
labels = []
for dir_ in tqdm(os.listdir(base_path)):
    if not dir_.startswith('.'):
        name = dir_.split('_')[0]
        for image in os.listdir(base_path + dir_):
            if not image.startswith('.'):
                img = Image.open(base_path + dir_ + '/' + image).convert('L')
                img.thumbnail((64,64))
                images.append(np.array(img))
                labels.append(name)

100%|██████████| 11498/11498 [00:31<00:00, 362.93it/s]


In [48]:
from collections import Counter

In [49]:
c = Counter(labels)

In [50]:
names_selected = [kv[0] for kv in c.items() if kv[1] > 1]

In [51]:
idx = [True if l in names_selected else False for l in labels]

In [52]:
images = np.array(images)/255
labels = np.array(labels)

In [53]:
images = images[idx, :, :]
labels = labels[idx]

In [54]:
num_labels = len(set(labels))

In [55]:
X_tr,X_te, y_tr,y_te = train_test_split(images, labels, test_size=0.1, random_state=0, stratify=labels)

In [56]:
from sklearn.preprocessing import MultiLabelBinarizer

In [57]:
binarizer = MultiLabelBinarizer().fit(np.array(list(set(labels))).reshape((-1,1)))

In [58]:
y_tr = binarizer.transform(y_tr.reshape((-1,1)))

In [60]:
labels = binarizer.transform(labels.reshape((-1,1)))

In [17]:
# del images, labels

In [61]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout, Input, Convolution2D, LeakyReLU
from keras.optimizers import Adam
from keras.activations import softmax, relu, sigmoid
from keras.metrics import binary_crossentropy, categorical_crossentropy
from keras.regularizers import L1L2Regularizer

In [62]:
# model = Sequential()
# model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=(128, 128, 3), activation=relu))
# model.add(Conv2D(32, 3, 3, border_mode='same', activation=relu))
# model.add(MaxPooling2D(strides=(3,3)))
# model.add(Conv2D(32, 3, 3, border_mode='same', activation=relu))
# model.add(Conv2D(32, 3, 3, border_mode='same', activation=relu))
# model.add(MaxPooling2D(strides=(3,3)))
# model.add(Flatten())
# model.add(Dense(4096, W_regularizer=L1L2Regularizer(l2=1e-3)))
# model.add(Activation('relu'))
# model.add(Dense(y_tr.shape[1]))
# model.add(Activation('softmax'))

In [71]:
model = Sequential()
model.add(Dense(512,input_dim=64*64))
model.add(LeakyReLU())
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(LeakyReLU())
model.add(Dropout(0.2))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [72]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_26 (Dense)                 (None, 512)           2097664     dense_input_10[0][0]             
____________________________________________________________________________________________________
leakyrelu_17 (LeakyReLU)         (None, 512)           0           dense_26[0][0]                   
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 512)           0           leakyrelu_17[0][0]               
____________________________________________________________________________________________________
dense_27 (Dense)                 (None, 512)           262656      dropout_17[0][0]                 
___________________________________________________________________________________________

In [73]:
model.compile(loss=categorical_crossentropy, #categorical_crossentropy
              optimizer='adam',
              metrics=['accuracy'])

In [74]:
model.fit(x=images.reshape((-1, 64*64)), y=labels, batch_size=128, nb_epoch=100, shuffle=True)

Epoch 1/100
12145/12145 [==============================] - 12s - loss: 6.4742 - acc: 0.0403     

In [78]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [75]:
# model.fit(x=X_tr.reshape((-1, 64*64)), y=y_tr, batch_size=128, nb_epoch=100, shuffle=True, validation_split=0.2)

In [82]:
# img_jan = Image.open('./../img_ja.jpg').convert('L')

In [83]:
# np.array(img_jan).shape

In [ ]:
# im = img_jan.crop(box=(1000, 0, 4032, 3024))

In [84]:
# im = img_jan.crop(box=(0, 0, 1152, 1152))

In [ ]:
# im.thumbnail((64,64))

In [ ]:
im

In [ ]:
# im = im.rotate(-90)

In [ ]:
im = np.array(im)/255

In [ ]:
im.shape

In [ ]:
# im = np.pad(im, pad_width=((1,0), (0,0), (0,0)), mode='constant', constant_values=0)

In [ ]:
proba = model.predict_proba(im.reshape((1, 64*64)))

In [ ]:
from collections import defaultdict

In [ ]:
names = defaultdict(list)
for key, val in sorted(list(zip(binarizer.classes_, *proba)), key=lambda x: x[1], reverse=True):
    names[round(val, 3)].append(key)

In [ ]:
names